### Ingest lap_times folder

In [3]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [4]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

lap_times_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), True),
        StructField("lap", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("milliseconds", IntegerType(), True),
    ]
)

In [6]:
lap_times_df = spark.read.schema(lap_times_schema).csv(
    f"{landing_folder_path}/{v_file_date}/lap_times"
)

##### Step 2 - Add ingestion meta data

In [7]:
lap_times_with_metadata_df = (
    add_ingestion_date(lap_times_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write to output to bronze container

In [8]:
lap_times_with_metadata_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.lap_times"
)